# PWM QuickStart

Physics World Model — 26 imaging modalities, 43+ solvers, one unified toolkit.

This notebook walks through install, environment check, forward simulation, reconstruction, and SharePack export.

In [ ]:
# Cell 1: Install + Doctor
!pip install -e packages/pwm_core -q
!pwm doctor

In [ ]:
# Cell 2: Compile CASSI template, forward pass, reconstruct
import yaml
import numpy as np
from pwm_core.graph.compiler import GraphCompiler
from pwm_core.graph.graph_spec import OperatorGraphSpec

# Load templates
with open("packages/pwm_core/contrib/graph_templates.yaml") as f:
    templates = yaml.safe_load(f)["templates"]

# Compile CASSI operator graph
spec = OperatorGraphSpec.model_validate(templates["cassi_sd_graph_v1"])
compiler = GraphCompiler()
op = compiler.compile(spec)

# Forward: simulate measurement
x_gt = np.random.RandomState(42).rand(*op.x_shape).astype(np.float32)
y = op.forward(x_gt)

# Reconstruct via adjoint (simple back-projection)
x_hat = op.adjoint(y)

print(f"Ground truth shape: {x_gt.shape}")
print(f"Measurement shape:  {y.shape}")
print(f"Reconstruction:     {x_hat.shape}")

In [ ]:
# Cell 3: Display images
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(x_gt, cmap="viridis")
axes[0].set_title("Ground Truth (x)")
axes[0].axis("off")

axes[1].imshow(y, cmap="viridis")
axes[1].set_title("Measurement (y)")
axes[1].axis("off")

axes[2].imshow(x_hat, cmap="viridis")
axes[2].set_title("Adjoint Reconstruction")
axes[2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Cell 4: Show all 26 modalities
with open("packages/pwm_core/contrib/modalities.yaml") as f:
    modalities = yaml.safe_load(f)

mod_list = modalities.get("modalities", modalities)
if isinstance(mod_list, dict):
    names = sorted(mod_list.keys())
else:
    names = [m.get("id", m.get("name", str(m))) for m in mod_list]

print(f"PWM supports {len(names)} imaging modalities:\n")
for i, name in enumerate(names, 1):
    print(f"  {i:2d}. {name}")

In [ ]:
# Cell 5: SharePack export via CLI
!pwm demo cassi --run --export-sharepack
print("\nSharePack exported! Check runs/demo_cassi/sharepack/")